# Atelier web avec Python

L'objectif de cet atelier est de travailler sur un cas concret. Il s'agit de trouver une solution à un petit problème et de le mettre en oeuvre avec Python, ce n'est pas une présentation générale du web avec Python ou même d'un framework.

Quel est ce problème ? Guersande a bossé sur les partitifs en finnois à partir du corpus [Europarl](http://www.statmt.org/europarl/). Le résultat de son travail est présenté dans une page web : [apps.lattice.cnrs.fr/partitif](http://apps.lattice.cnrs.fr/partitif)  
Rien à dire sur le travail mené, ni même sur la présentation des résultats mais la page web générée pèse 319 Mo 😨😨😨. C'est trop, beaucoup trop.  
Notre problème à résoudre est : **comment réduire la taille du fichier html sans trop modifier sa structure et sa présentation ?**

Une solution aurait pu être de paginer les résultats, ce qui reviendrait à avoir plusieurs pages html statiques au lieu d'une. C'est facile à mettre en oeuvre et cela résoud le problème mais l'utilisateur perd la vue sur l'intégralité des résultats. Et puis avec cette solution ce notebook n'aurait pas lieu d'être.

Celle que l'on va essayer de mettre en oeuvre ne change rien pour l'utilisateur : il a tous les lemmes sur une seule page et il peut afficher les exemples à la demande.  
Les exemples c'est précisement là-dessus que l'on va opérer le changement. Dans la version actuelle les exemples sont cachés/montrés avec le clic de l'utilisateur mais ils sont inclus dans le code html, c'est même eux qui sont en grande partie responsables de la taille de la page. L'idée ici est de les extraire de la page afin de l'alléger et de les inclure uniquement lors du clic de l'utilisateur.

## Extraction des exemples

Le premier truc qu'on va devoir faire sur la page est d'extraire les exemples de la page web et les stocker autre part, on verra où après.

Pour s'exercer on va travailler sur un fichier réduit, on passera au gros fichier une fois que tout sera au point. Le fichier de travail est `test.html`.

Je donne la trame, à vous de compléter : 

In [ ]:
from bs4 import BeautifulSoup
import json


soup = BeautifulSoup(open("test.html"), 'html.parser')
divs = soup.find_all('div', class_="structure")
    
verbs = dict()
for div in divs:                                  
    verbs[div['id']] = list()               
    examples = div.find_all('div', class_="exemple")
    print("{} examples divs found for verb {}".format(len(examples), div['id']))


## HTML + JSON : et maintenant ?

On a une page html avec une taille décente d'un côté (476 Ko) et un fichier json (230 Mo) avec les exemples de l'autre. Super. Notez qu'un passage on a bien élagué en supprimant le balisage html.

Maintenant ben il va falloir adapter le "voir/masquer les exemples" de la page originale, le rendre dynamique en incluant le code html des exemples à la volée. Ça se fait en Javascript grâce à la techno [AJAX](https://fr.wikipedia.org/wiki/Ajax_%28informatique%29), et pour se simplifier la vie on va utiliser la bibliothèque JQuery.

### Modification du DOM

Pour jouer avec JS le plus pratique est d'utiliser les outils inclus dans le navigateur. Dans Chrome ça s'appelle 'Developper tools' (ctrl+maj+I ou clic droit 'Inspect'), allez dans l'onglet 'Sources', ctrl+P et sélectionnez le fichier à éditer. Pour nous ce sera `flipflop.js`

Deux fonctions nous intéressent dans ce fichier :
* `flipflipON` est appelée par un clic sur 'voir les exemples'
* `flipflopOFF` est appelée par un clic sur 'cacher les exemples'

Nous allons modifier la fonction `flipflopON` et utliser la fonction JQuery `append` pour ajouter du contenu à l'élément sélectionné. Modifiez le code directement dans le naivigateur pour voir ce que ça donne.

```javascript
function flipflopON(id1,id2)
{
if (document.getElementById(id1).style.display == "none") {
document.getElementById(id1).style.display = "block";
document.getElementById(id2).style.display = "none";
$('#'+id1).append("<div class=\"exemple\">Youhou !</div>")
}
else    {document.getElementById(id1).style.display = "none";
document.getElementById(id2).style.display = "block";
}
}
```

C'est cool ça marche mais si vous "voir/masquer" plusieurs fois vous verrez que les éléments ajoutés sont conservés. Il va falloir modifier aussi la fonction `flipflopOFF` pour faire le ménage.

```javascript
function flipflopOFF(id1,id2)
{
if (document.getElementById(id1).style.display == "none") {
document.getElementById(id1).style.display = "block";
document.getElementById(id2).style.display = "none";
$('#'+id2+">div.exemple").remove();
}
else	{document.getElementById(id1).style.display = "block";
document.getElementById(id2).style.display = "none";
}
}
```

### Conversion du json en html

C'est bien c'est bien mais nous on a du json en entrée. Il nous faut une fonction qui transforme du json dans l'html désiré. Au travail.

Voici un exemple d'entrée JSON pour 3 exemples du verbe 'keskittää' :

```json
[{"sp": " keskittää + keskustelu (Nom\n)", "phrase": "Olisi siis virhe keskittää Euroopan parlamentin vuoden 2001 talousarviota koskeva keskustelu pelkästään tähän kysymykseen. "},{"sp": " keskittää + kaikki (Nom\n)", "phrase":"Keskititte kaikki tietenkin puheenvuoronne rauhanprosessiin ja neuvotteluihin, jotka ovat epäilemättä edistymisen avaintekijöitä tällä Lähi-idän alueella. "}, {"sp": " keskittää + puheen#vuoro (Nom\n)", "phrase":"Keskititte kaikki tietenkin puheenvuoronne rauhanprosessiin ja neuvotteluihin, jotka ovat epäilemättä edistymisen avaintekijöitä tällä Lähi-idän alueella. "}]
```

Qui devra donner en html :

In [ ]:
%%html
<div class="exemple">
<div class="sp">keskittää + keskustelu (Nom\n)</div>
<div class="phrase">Olisi siis virhe keskittää Euroopan parlamentin vuoden 2001 talousarviota koskeva keskustelu pelkästään tähän kysymykseen.</div>
</div>

Pour cela nous allons utiliser la fonction [each](http://api.jquery.com/jquery.each/) de JQuery qui permet d'itérer sur des objets ou des tableaux.

Copiez-collez dans Developper Tools dans le navigateur et regardez ce que ça donne.

```javascript
function flipflopON(id1,id2)
{
examples = [{"sp": " keskittää + keskustelu (Nom\n)", "phrase": "Olisi siis virhe keskittää Euroopan parlamentin vuoden 2001 talousarviota koskeva keskustelu pelkästään tähän kysymykseen. "},{"sp": " keskittää + kaikki (Nom\n)", "phrase":"Keskititte kaikki tietenkin puheenvuoronne rauhanprosessiin ja neuvotteluihin, jotka ovat epäilemättä edistymisen avaintekijöitä tällä Lähi-idän alueella. "}, {"sp": " keskittää + puheen#vuoro (Nom\n)", "phrase":"Keskititte kaikki tietenkin puheenvuoronne rauhanprosessiin ja neuvotteluihin, jotka ovat epäilemättä edistymisen avaintekijöitä tällä Lähi-idän alueella. "}]

if (document.getElementById(id1).style.display == "none") {
document.getElementById(id1).style.display = "block";
document.getElementById(id2).style.display = "none";

$.each(examples, function(index, example){
    ex_elem = $('<div/>', {'class': 'exemple'}).append(
        $('<div/>', {'class': 'sp', 'text': example.sp}),
        $('<div/>', {'class': 'phrase', 'text': example.phrase})
    );
   $('#'+id1).append(ex_elem); 
});
}
else	{document.getElementById(id1).style.display = "none";
document.getElementById(id2).style.display = "block";
}
}
```

À ce stade nous sommes capables d'afficher les exemples à partir d'une donnée JSON : parcourir le JSON, générer le code html pour emballer les données et modifier le DOM. Tout ça piloté par les clics de l'utilisateur.

### Service Python
On avance mais notre exemple fonctionne avec une donnée en dur. Ce qu'il nous faut maintenant c'est récupérer le tableau d'exemples correspondants à un verbe donné. Cela signifie :
1. Charger le fichier `verbs.json`, trouver le tableau des exemples pour un verbe donné et le renvoyer. Fonction Python, facile.
2. "Servir" le résultat en json. Avec Flask, pas bien compliqué.
3. Appeler le serveur flask en JS côté client

Un peu à l'arrache on peut faire un truc comme ça avec Flask : 

In [ ]:
from flask import Flask, jsonify
import json

examples = json.load(open("examples.json"))

app = Flask(__name__)

@app.route('/<verb>')
def hello_world(verb):
    
    if verb in examples:
        res = examples[verb]
    else:
        res = ""
    return jsonify(res)

if __name__ == '__main__':
    app.run()

C'est tout ? Oui (enfin presque). Essayez avec les url http://localhost:5000/pilata, http://localhost:5000/keskittää, http://localhost:5000/machin  
Ce n'est pas vraiment la solution idéale, dans la vraie vie on pourra utiliser [redis](https://redis.io/) pour éviter de charger les 230 Mo de json en mémoire avec Flask mais ça ajoute (encore) une techno. Ça se discute, c'est selon les compétences et les habitudes du dév.

### Et côté client ?

JQuery nous facilite bien la vie, encore une fois on y trouve la fonction qu'il nous faut : [ajax](http://api.jquery.com/jquery.ajax/)

```javascript
function flipflopON(id1,id2)
{
    if (document.getElementById(id1).style.display == "none") {
	document.getElementById(id1).style.display = "block";
	document.getElementById(id2).style.display = "none";
	verb = id1.substring(7) //contenupilata -> pilata
	
	$.ajax({
	    url: 'http://localhost:5000/'+verb,
	    type: "GET",
	    success: function(examples){
		$.each(examples, function(index, example){
		    ex_elem = $('<div/>', {'class': 'exemple'}).append(
			$('<div/>', {'class': 'sp', 'text': example.sp}),
			$('<div/>', {'class': 'phrase', 'text': example.phrase})
		    );      
		    $('#'+id1).append(ex_elem); 
		});
		
	    }
	});
    }
    else{
	document.getElementById(id1).style.display = "none";
	document.getElementById(id2).style.display = "block";
    }
}
```

Et ça marche !!

Ça marche en local en tout cas. Le plus compliqué dans tout ça est sans doute le déploiement sur un serveur de production. Flask ne **doit pas** être utilisé en prod directement, voir http://flask.pocoo.org/docs/0.12/deploying/#deployment  
Si on a la main sur le serveur Gunicorn est une option fiable et facile à mettre en oeuvre.  
Il reste d'autres questions relatives au déploiement mais elles sont un peu hors sujet ici.